# ZOO: Classificação Versão 2

* Conjunto de treinamento: 20%   
* Conjunto de teste: 80%

In [0]:
import numpy as np
import pandas as pd
from sklearn import datasets

In [9]:
from sklearn import datasets
import io
from google.colab import files 


uploadedFile = files.upload()

Saving zoo.data to zoo (3).data


In [10]:
colunas_zoo = ['Nome do Animal','Cabelo','Penas','Ovos','Leite','No ar','Aquático','Predador','Dentado', 'Espinha Dorsal', 'Respira', 'Venenoso', 'Barbatanas', 'Patas', 'Cauda', 'Domesticado', 'Tamanho', 'Tipo']
df_zoo = pd.read_csv(io.StringIO(uploadedFile['zoo.data'].decode('utf-8')),names=colunas_zoo,header=None) 
#df_zoo = pd.read_csv(io.StringIO(uploadedFile['zoo.data'].decode('utf-8')),header=None)

linhas, colunas = df_zoo.shape
tamanho_treino = int(np.ceil(linhas*.20))

xtreinozoo = df_zoo.iloc[:tamanho_treino,1:colunas-1]
ytreinozoo = df_zoo.iloc[:tamanho_treino,colunas-1:]
xtestezoo = df_zoo.iloc[tamanho_treino:,1:colunas-1]
ytestezoo = df_zoo.iloc[tamanho_treino:,colunas-1:]

df_zoo.head(5)

,Nome do Animal,Cabelo,Penas,Ovos,Leite,No ar,Aquático,Predador,Dentado,Espinha Dorsal,Respira,Venenoso,Barbatanas,Patas,Cauda,Domesticado,Tamanho,Tipo
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


#REGRESSÃO LOGÍSTICA

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

regressao_logistica = LogisticRegression()

regressao_logistica.fit(xtreinozoo, ytreinozoo)
previstos_reg_logistica = regressao_logistica.predict(xtestezoo)

confusion_matrix(ytestezoo,previstos_reg_logistica)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[31,  0,  0,  0,  0,  0,  0],
       [ 0, 17,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  3,  0,  0,  0],
       [ 0,  0,  0,  8,  0,  0,  0],
       [ 4,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  8],
       [ 1,  2,  0,  0,  0,  0,  4]])

#KNN

In [12]:
from sklearn import neighbors
from sklearn.metrics import classification_report

knn = neighbors.KNeighborsClassifier(n_neighbors=5)

knn.fit(xtreinozoo, ytreinozoo)
previstos_knn = knn.predict(xtestezoo)

print(classification_report(ytestezoo,previstos_knn))

             precision    recall  f1-score   support

          1       0.62      0.94      0.74        31
          2       1.00      1.00      1.00        17
          3       0.00      0.00      0.00         5
          4       0.50      1.00      0.67         8
          5       0.00      0.00      0.00         4
          6       0.00      0.00      0.00         8
          7       0.00      0.00      0.00         7

avg / total       0.50      0.68      0.57        80



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#Rede Neural: FEEDFORWARD

In [13]:
from keras.models import Sequential 
from keras.layers.core import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

features = np.size(xtreinozoo,1)

def criar_modelo():
  model = Sequential()
  model.add(Dense(50, input_dim=features, activation='sigmoid'))
  model.add(Dense(30, activation='sigmoid'))
  model.add(Dense(1, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

rede_neural_keras = KerasClassifier(build_fn=criar_modelo,verbose=0)

rede_neural_keras.fit(xtreinozoo, ytreinozoo, epochs=5, batch_size=10)

previstos_keras = rede_neural_keras.predict(xtestezoo)

confusion_matrix(ytestezoo,previstos_keras)

array([[ 0, 31,  0,  0,  0,  0,  0],
       [ 0, 17,  0,  0,  0,  0,  0],
       [ 0,  5,  0,  0,  0,  0,  0],
       [ 0,  8,  0,  0,  0,  0,  0],
       [ 0,  4,  0,  0,  0,  0,  0],
       [ 0,  8,  0,  0,  0,  0,  0],
       [ 0,  7,  0,  0,  0,  0,  0]])

#K-MEANS

In [14]:
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix

kmeans = KMeans(n_clusters=3)

kmeans.fit(xtreinozoo)
previstos_kmeans = kmeans.predict(xtestezoo)

confusion_matrix(ytestezoo,previstos_kmeans)

array([[ 0,  0,  0,  0,  0,  0,  0,  0],
       [27,  2,  2,  0,  0,  0,  0,  0],
       [ 0,  0, 17,  0,  0,  0,  0,  0],
       [ 2,  3,  0,  0,  0,  0,  0,  0],
       [ 0,  8,  0,  0,  0,  0,  0,  0],
       [ 4,  0,  0,  0,  0,  0,  0,  0],
       [ 8,  0,  0,  0,  0,  0,  0,  0],
       [ 4,  3,  0,  0,  0,  0,  0,  0]])